# Computer Vision and Convolutional Neural Networks

In 2012, Alex Krizhevsky et al. competed in the [ImageNet Large Scale Visual Recognition Challenge](https://arxiv.org/abs/1409.0575). Their solution, based on a deep _Convolutional Neural Network_ (CNN) beat the runner up by more than 10% ([their paper](https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf)). Prior to this result, many thought that it was infeasable to train such deep models due to the large number parameters that need to be optimized and the associated high computational cost. Krizhevsky et al. were among the first to user GPUs to speed things up. This landmark result kicked off a wave of innovation that continues to this day. 

Convolutional Neural Networks are named thus because of the _convolution_ operation which plays a primary role in their design. You can take a look at [these slides](https://docs.google.com/presentation/d/17Uygrr1g6qBMpXOwcfZf584PkW2LTZoXgX1mqsK6syE/edit?usp=sharing) to get a sense for what the convolution operator does. Convolution is a great example of building _inductive biases_ into the network architecture. In this case, the inductive priors are _locality_ and _translation invariance_. These assumptions lead to weight sharing which significantly improves the parameter efficiency of our model relative to the fully connected neural networks that we've seen so far. 

In this notebook, we apply CNNs to the classification of handwritten characters (letters and numbers) using the [EMNIST dataset](https://www.westernsydney.edu.au/icns/reproducible_research/publication_support_materials/emnist).  The EMNIST dataset has the digits 0 through 9 and a total of 37 upper and lower-case latin characters (see Figure) for a total of 47 classes. The latin characters make EMNIST a more difficult classification task than the more famous MNIST dataset which only contains digits. We will work through a variety of architecture design considerations, will train the model for each case, and learn to reason about how architecture choices impact the performance characteristics of the model.

![Balanced EMNIST Dataset](https://www.researchgate.net/publication/329391937/figure/fig3/AS:700515201605633@1544027247072/Visualization-of-EMNIST-balanced-dataset-10-Fig2.ppm)

# Setup

In [ ]:
# Library imports
import os
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from PIL import Image
import math
%matplotlib inline

# set random seed for reproducibility
_ = torch.manual_seed(0)

For more information about Palmetto storage, see the [onboarding training](https://docs.rcd.clemson.edu/palmetto/onboarding/) and the documentation for [Palmetto storage](https://docs.rcd.clemson.edu/palmetto/storage/store/).

In [ ]:
# save locations
# best not to use your home directory for reading/writing large temporary files
# /scratch is much faster!
data_dir = f"/scratch/{os.environ['USER']}/data"
model_path = f"/scratch/{os.environ['USER']}/model.pt"

# Model and Training
batch_size=64 #input batch size for training (default: 64)
test_batch_size=1000 #input batch size for testing (default: 1000)
num_workers=9 # parallel data loading to speed things up
lr=1.0 # learning rate 
gamma=0.7 # Learning rate step gamma (default: 0.7)
no_cuda=False #disables CUDA training (default: False)
seed=42 #random seed (default: 42)
log_interval=10 #how many batches to wait before logging training status (default: 10)
save_model=False #save the trained model (default: False)

# additional derived settings
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

print("Device:", device)

In [ ]:
# Loading the data

# Pytorch provides a number of pre-defined dataset classes
# EMNIST is one of them! Pytorch will automatically download the data.
# It will only download if the data is not already present.
data_train = datasets.EMNIST(data_dir, split='balanced', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.1307,), (0.3081,))
                                 ]))

data_test = datasets.EMNIST(data_dir, split='balanced', train=False, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                                ]))

# define pytorch dataloaders for training and testing
train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=test_batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

# save a test batch for later testing
image_gen = iter(test_loader)
test_img, test_trg = next(image_gen)

In [ ]:
print("Training dataset:", train_loader.dataset)
print("Testing dataset:", test_loader.dataset)

In [ ]:
# Look at some examples
n_images = 16
rand_imgs = next(iter(test_loader))
ints = torch.randint(0,len(rand_imgs[0]), (n_images,))

fig, ax_arr = plt.subplots(4, 4)
ax_arr = ax_arr.flatten()

for n, ix in enumerate(ints):
    img = rand_imgs[0][ix]
    ax_arr[n].imshow(img[0].detach().cpu().numpy().T, cmap='Greys')
    ax_arr[n].axis('off')
plt.tight_layout()
plt.show()



---

**Question**: In addition to the larger number of classes in EMNIST compared to MNIST, why might it be more difficult to accurately classify images from the EMNIST set?

---


In [ ]:
# Training and testing functions
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('\r\tTrain epoch {}: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()), end='')
            
def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\rTest epoch {}: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
def train_and_test(model, save_name=model_path, epochs=5):
    # Train the linear model
    optimizer = optim.Adadelta(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader, epoch)
        scheduler.step()

    if save_model:
        torch.save(model.state_dict(), save_name)

def get_n_params(model): return sum(p.numel() for p in model.parameters())

# Classification architectures
We will explore a range of classification architectures starting with a linear classifier and scaling up to a several-layer convolutional neural network. Each architecture can be described in two steps (see figure below):
1. Feature extraction -- converting the raw pixel information into a _feature vector_
2. Classification -- converting the feature vector into predictions about the character in the input image.

![CNN diagram](https://miro.medium.com/max/1400/1*XbuW8WuRrAY5pC4t-9DZAQ.jpeg)

Most of the secret sauce of CNNs takes place in the feature extraction step. We will define a generic classification architecture that takes as input the sequence of operations used for feature extraction. We will use progressively more complex feature extraction pipelines. For the classification step, we will always use a single linear layer, but the number of inputs will depend on the size of the feature vector produced by the feature extractor. 

The class defined below implements this generic classifier class in pytorch. 

In [ ]:
# Define the generic EMNIST classification architecture
# The `feature_extractor` argument takes images as input and produces feature vectors.
# These vectors can be of any length. 
class Classifier(nn.Module):
    def __init__(self, feature_extractor):
        super(Classifier, self).__init__()

        # feature encoder
        self.feature_extractor = feature_extractor
        
        # classifier
        # LazyLinear allows input size to be inferred automatically 
        self.classifier = nn.LazyLinear(47) # 47 classes in EMNIST

    def forward(self, x):
        x = self.feature_extractor(x) # Flatten -> x_i
        x = self.classifier(x) # Matrix multiply -> c_m^0 + sum(W_mi*x_i)
        x = F.softmax(x, dim=1) # apply the softmax function
        return x

# Linear Classifier

A linear classifier uses the original pixels as the features for classification. We simply need to unravel the 2D image into a vector. This is accomplished using `nn.Flatten()`. 

Note: you can safely ignore the message `UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development`

In [ ]:
# set random seed for reproducibility
torch.manual_seed(0)

# define the feature extractor
# for linear model, we just flatten the original image into a vector
feature_extractor = nn.Flatten()

# Create the model
model = Classifier(feature_extractor).to(device)

In [ ]:
# Display model and number of parameters 
# Run one batch through the model to initialize the lazy linear layer
# this is necessary to get accurate parameter counts
with torch.no_grad():
    model(test_img.to(device)) 

display(model)

print("Number of parameters in linear model:", get_n_params(model))

## Train and test the linear model

In [ ]:
train_and_test(model)

# by default, `train_and_test` trains for 5 epochs
# you can adjust this using the epochs argument, like
# train_and_test(model, epochs=2)

---

**Question**: Think of a naive performance baseline for this task. How well does it compare with that performance? Do you find this suprising given that this is just a linear model?  

---

# CNN Classifier

### One convolution layer
We now make a more interesting feature extractor using convolution. The feature extractor has the following steps: 
1. _convolution_ (`nn.Conv2d`) over the input image.
2. _batch normalization_ (`nn.BatchNorm2d`) rescales the feature maps. This helps the network converge more quickly. You can see the definition [here](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html).
3. _max pooling_ (`nn.MaxPool2d`) takes the maximum pixel from each 2x2 block resulting in an image that has half the size in each dimension. 
4. _ReLU_ (`nn.ReLU`) zeroes out negative values in the feature maps. See figure:

![ReLU](https://www.researchgate.net/profile/Hossam-H-Sultan/publication/333411007/figure/fig7/AS:766785846525952@1559827400204/ReLU-activation-function.png)

In [ ]:
# set random seed for reproducibility
torch.manual_seed(0)

# define the feature extractor
feature_extractor = nn.Sequential(
    # convolution block:
    nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, bias=False),
    nn.LazyBatchNorm2d(),
    nn.ReLU(),
    nn.MaxPool2d(2),
    # flatten just as with the linear classifier
    nn.Flatten()
)
# Create the model
model = Classifier(feature_extractor).to(device)

In [ ]:
# @title Display model and number of parameters 
# Run one batch through the model to initialize the lazy linear layer
# this is necessary to get accurate parameter counts
with torch.no_grad():
    model(test_img.to(device)) 

display(model)

print("Number of parameters in linear model:", get_n_params(model))

(Less than 36,895 for linear model)

---

**Question**: Why does the CNN have fewer parameters than the linear model?

---

In [ ]:
train_and_test(model)

---

**Question**: Why does the CNN beat the linear model after 5 epochs despite having fewer parameters?

---

**Feature maps:** let's look at the internal representations of the images produced by the 4 convolution kernel. In the picture below, the colored grids on the left are representations of the convolution kernel matrices. The values of these kernel matrices were "learned" during the training of the network. The image on the right is the feature map produced by convolving the kernel matrix with the original image. Bright areas correspond to high numerical values; dark areas correspond to low numerical values. Rerun the cell to see the feature maps for different images.


In [ ]:
imgs, labels = next(image_gen)
test_img_plot = imgs[6,0]

kern = model.feature_extractor[0]

# Generate feature maps
fmaps_1 = kern(test_img_plot[None, None].cuda()).detach().cpu().numpy()[0]
for K, fmap in zip(kern.weight, fmaps_1):
    # plot kernel
    plt.subplot(1,2,1)
    mat = K.clone().detach().cpu().numpy()[0]
    plt.imshow(mat.T)
    plt.axis('off')

    # plot feature map
    plt.subplot(1,2,2)
    plt.imshow(fmap.T, cmap='Greys')
    plt.axis('off')
    plt.show()

# A "wider" model
The "width" of a CNN refers to how many feature maps are used in the convolution layers. Wide layers have many convolution kernels leading to many feature maps. In the model above, we had 4 convolution kernels (see `out_channels=4` in the `nn.Conv2d` layer in the feature extractor). Interpreting these kernels as feature extractors, it's plausible that we could improve the performance of the model by increasing the number of kernels.

The network below is the same as the previous one except that it uses twice as many kernels. 

In [ ]:
# set random seed for reproducibility
torch.manual_seed(0)

# define the feature extractor
feature_extractor = nn.Sequential(
    # convolution block:
    nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, bias=False),
    nn.LazyBatchNorm2d(),
    nn.ReLU(),
    nn.MaxPool2d(2),
    # flatten just as with the linear classifier
    nn.Flatten()
)

# Create the model
model = Classifier(feature_extractor).to(device)

In [ ]:
# @title Display model and number of parameters 
# Run one batch through the model to initialize the lazy linear layer
# this is necessary to get accurate parameter counts
with torch.no_grad():
    model(test_img.to(device)) 

display(model)

print("Number of parameters in linear model:", get_n_params(model))

Notice that doubling the number of channels nearly doubled the number of free parameters in the model. 

In [ ]:
train_and_test(model)

---

**Question**: How does the performance with 8 kernels compare to the performance with 4 kernels? Why do you think this might be the case?

---

# A "deeper" model


The "depth" of a CNN refers to how many convolution layers are used in the network. The feature maps from one convolution step are used as the input for the next convolution. The previous network had a single convolution step. We will now try adding more convolution layers and see how it effects performance. To isolate the effect of having more layers, we will hold the width at 4 kernels like our first CNN.

---

**Question**: Why might it be helpful to add more convolutional layers to the CNN? Can you contrast the effect of adding more depth to the effect of greater width?

---

In [ ]:
# set random seed for reproducibility
torch.manual_seed(0)

# define the encoder
# we repeat the convolution block several times
feature_extractor = nn.Sequential(
    # block 1
    nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, bias=False),
    nn.LazyBatchNorm2d(),
    nn.ReLU(),
    # 2
    nn.Conv2d(in_channels=4, out_channels=4, kernel_size=3, stride=1, bias=False),
    nn.LazyBatchNorm2d(),
    nn.ReLU(),
    # 3
    nn.Conv2d(in_channels=4, out_channels=4, kernel_size=3, stride=1, bias=False),
    nn.LazyBatchNorm2d(),
    nn.ReLU(),
    nn.MaxPool2d(2),
    # flatten just as with the linear classifier
    nn.Flatten()
)

# Create the model
model = Classifier(feature_extractor).to(device)

In [ ]:
# @title Display model and number of parameters 
# Run one batch through the model to initialize the lazy linear layer
# this is necessary to get accurate parameter counts
with torch.no_grad():
    model(test_img.to(device)) 

display(model)

print("Number of parameters in linear model:", get_n_params(model))

---

**Question**: The number of parameters is actually _less_ than our single layer model. We added lots of convolution kernels, so how can that be?

---

In [ ]:
train_and_test(model)

---

**Question**: You should find that the deeper model performed similarly to the wider model despite having many fewer parameters. Why do you think increasing the depth of the model was more parameter efficient than increasing the width?

---

# Your turn! 
Modify the feature extractor below and see how good of a score you can obtain on EMNIST. Some ideas: 
* Change the number of channels in the convolution layers. Note: the `in_channels` argument to `nn.Conv2d` must match the `out_channels` argument from a previous layer. The input image has 1 channel since the images are grayscale.
* Change the `kernel_size`, which is the size of the kernel matrix used in convolution. 
* Change the number of convolution blocks in the network.
* Try [different activation functions](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity).
* Add or remove max pooling layers between convolution blocks
* Use a greater number of epochs in the `train_and_test` function
* Advanced: can you figure out how to add "residual"/"skip" connections? See [this paper](https://arxiv.org/abs/1512.03385). 

There are many, many possible combinations that you could try, and you can't explore them all. Try to think of a few experiments to try and record what you learn. If you want to go down the rabbit hole of trying many things, feel free, but that's not expected for this assignment.

Try to beat my score of ~80% resulting from using the "deeper model" above trained for 13 epochs.

In [ ]:
# set random seed for reproducibility
torch.manual_seed(0)

# define the encoder
# we repeat the convolution block several times
feature_extractor = nn.Sequential(
    # block 1
    nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, stride=1, bias=False),
    nn.LazyBatchNorm2d(),
    nn.ReLU(),
    # 2
    nn.Conv2d(in_channels=4, out_channels=4, kernel_size=3, stride=1, bias=False),
    nn.LazyBatchNorm2d(),
    nn.ReLU(),
    # 3
    nn.Conv2d(in_channels=4, out_channels=4, kernel_size=3, stride=1, bias=False),
    nn.LazyBatchNorm2d(),
    nn.ReLU(),
    nn.MaxPool2d(2),
    # flatten just as with the linear classifier
    nn.Flatten()
)

# Create the model
model = Classifier(feature_extractor).to(device)

In [ ]:
train_and_test(model, epochs=13)